In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
## load the ANN trained model, scaler, pickle, onehot
model = load_model('model.h5')

with open('labe_encoder_gender.pkl','rb') as file:
    label_encoder_gender= pickle.load(file)

with open('onehot_encoder_geo.pkl','rb') as file:
    label_encoder_geo= pickle.load(file)
    
with open('scaler.pkl','rb') as file:
    scaler= pickle.load(file)

In [3]:
# Example input data

input_data = { 'CreditScore':600,
                'Geography':'France',
                'Gender':'Male',
                'Age':40,
                'Tenure': 3,
                'Balance': 60000,
                'NumOfProducts': 2,
                'HasCrCard': 1,
                'IsActiveMember': 1,
                'EstimatedSalary': 50000}

In [4]:
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

In [5]:
# One-hot encode 'Geography'
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

In [6]:
input_data = pd.DataFrame([input_data])
input_data

In [7]:
input_df = pd.concat([input_data.reset_index(drop=True), geo_encoded_df], axis=1)
input_df

In [8]:
## Encode categorical variables

input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

In [9]:
input_df = input_df.drop("Geography",axis=1)
input_df

In [10]:
## sacling the input data
input_scaled = scaler.transform(input_df)

In [11]:
prediction = model.predict(input_scaled)

In [13]:
prediction_proba = prediction[0][0]
prediction_proba

In [14]:
if prediction_proba > 0.5:
    print('The customer is likely to churn')
else:
    print('The customer is not likely to churn')